This notebook runs a small test of the di-SNE implementation, visualizing a subset of the [Iris Dataset example](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html#sphx-glr-auto-examples-datasets-plot-iris-dataset-py), which can be found at the scikit-learn website. The example dataset is included in this folder as `test_dataset.txt`.

```
# install packages
pip install --user scanpy leidenalg
```

In [20]:
# set up packages to cluster data
import sys
import os
sys.path.append(os.environ["HOME"]+"/.local/lib/python3.9/site-packages")

In [21]:
import scanpy as sc, anndata as ad
import leidenalg
import numpy as np
import pandas as pd

In [27]:
# diSNE packages 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [22]:
# check imports
sc.logging.print_versions()

-----
anndata     0.9.1
scanpy      1.9.3
-----
PIL                 8.3.1
anyio               NA
attr                21.2.0
babel               2.9.1
backcall            0.2.0
beta_ufunc          NA
binom_ufunc         NA
bottleneck          1.3.2
brotli              NA
certifi             2022.12.07
cffi                1.14.6
chardet             4.0.0
charset_normalizer  2.0.0
cloudpickle         1.6.0
colorama            0.4.6
cycler              0.10.0
cython_runtime      NA
cytoolz             0.11.0
dask                2021.06.2
dateutil            2.8.2
debugpy             1.6.7
decorator           5.0.9
defusedxml          0.7.1
entrypoints         0.3
fastjsonschema      NA
fsspec              2021.07.0
google              NA
h5py                3.3.0
idna                3.1
igraph              0.11.4
ipykernel           6.17.1
ipython_genutils    0.2.0
ipywidgets          7.6.3
jedi                0.18.0
jinja2              3.0.1
joblib              1.0.1
json5               N

In [8]:
# read in example dataset as AnnData object
dataset = ad.read_csv('test_dataset.txt', delimiter='\t')

In [15]:
dataset.var

""
sepal_length
sepal_width
petal_length
petal_width
species


In [16]:
# compute neighborhood graphs, needed to run clustering
sc.pp.neighbors(dataset)

In [23]:
# clusters cells based on expression profiles, this is needed to color cells by cluster
sc.tl.leiden(dataset)

In [25]:
dataset

AnnData object with n_obs × n_vars = 5 × 5
    obs: 'leiden'
    uns: 'neighbors', 'leiden'
    obsp: 'distances', 'connectivities'

In [24]:
# TEST: use diSNE to run t-SNE and visualize clusters
# Step 1: Compute pairwise distances
def pairwise_distances(X):
    return squareform(pdist(X, 'euclidean'))

# Step 2: Compute joint probabilities P in high-dimensional space
def compute_joint_probabilities(distances, perplexity=30.0):
    (n, _) = distances.shape
    P = np.zeros((n, n))
    beta = np.ones((n, 1))
    logU = np.log(perplexity)
    
    for i in range(n):
        betamin = -np.inf
        betamax = np.inf
        Di = distances[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))]
        H, thisP = Hbeta(Di, beta[i])
        
        Hdiff = H - logU
        tries = 0
        while np.abs(Hdiff) > 1e-5 and tries < 50:
            if Hdiff > 0:
                betamin = beta[i].copy()
                if betamax == np.inf:
                    beta[i] *= 2
                else:
                    beta[i] = (beta[i] + betamax) / 2
            else:
                betamax = beta[i].copy()
                if betamin == -np.inf:
                    beta[i] /= 2
                else:
                    beta[i] = (beta[i] + betamin) / 2
            
            H, thisP = Hbeta(Di, beta[i])
            Hdiff = H - logU
            tries += 1
        
        P[i, np.concatenate((np.r_[0:i], np.r_[i+1:n]))] = thisP
    
    P = P + np.transpose(P)
    P = P / np.sum(P)
    P = np.maximum(P, 1e-12)
    return P

def Hbeta(D, beta):
    P = np.exp(-D * beta)
    sumP = np.sum(P)
    H = np.log(sumP) + beta * np.sum(D * P) / sumP
    P = P / sumP
    return H, P

# Step 3: Initialize embedding Y
def initialize_embedding(n, dim=2):
    return np.random.randn(n, dim)

# Step 4: Compute low-dimensional affinities Q
def compute_low_dim_affinities(Y):
    distances = pairwise_distances(Y)
    inv_distances = 1 / (1 + distances)
    np.fill_diagonal(inv_distances, 0)
    Q = inv_distances / np.sum(inv_distances)
    Q = np.maximum(Q, 1e-12)
    return Q

# Step 5: Compute gradients and update embedding
def compute_gradients(P, Q, Y):
    n, dim = Y.shape
    dY = np.zeros((n, dim))
    PQ_diff = P - Q
    for i in range(n):
        dY[i, :] = 4 * np.sum((PQ_diff[:, i] * (Y[i, :] - Y)), axis=0)
    return dY

def update_embedding(Y, dY, learning_rate=200.0):
    return Y - learning_rate * dY

# Step 6: The main t-SNE function
def tsne(X, perplexity=30.0, n_iter=1000, learning_rate=200.0):
    distances = pairwise_distances(X)
    P = compute_joint_probabilities(distances, perplexity)
    Y = initialize_embedding(X.shape[0])
    
    for iter in range(n_iter):
        Q = compute_low_dim_affinities(Y)
        dY = compute_gradients(P, Q, Y)
        Y = update_embedding(Y, dY, learning_rate)
        
        if iter % 100 == 0:
            cost = np.sum(P * np.log(P / Q))
            print(f"Iteration {iter}: cost = {cost}")
    
    return Y

In [34]:
values = dataset.to_df().values
values

array([[5.1, 3.5, 1.4, 0.2, 0. ],
       [4.9, 3. , 1.4, 0.2, 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. ],
       [5. , 3.6, 1.4, 0.2, 0. ]], dtype=float32)

In [36]:
tsne_out = tsne(values, perplexity=30.0, n_iter=1000)

ValueError: operands could not be broadcast together with shapes (5,) (5,2) 

In [ ]:
# Assuming you have a 'cluster' column in your original data
# i think the 'leiden' col is equivalent to cluster col?
# Add t-SNE results and cluster labels to a new DataFrame for plotting
tsne_df['Cluster'] = data['cluster']

In [ ]:
import matplotlib.pyplot as plt

def plot_tsne_results(Y, labels=None, title='t-SNE Visualization'):
    """
    Plot the t-SNE results.

    Parameters:
    - Y: np.ndarray, the t-SNE embedding, shape (n_samples, 2)
    - labels: np.ndarray or list, cluster labels for the data points, shape (n_samples,)
    - title: str, title of the plot
    """
    plt.figure(figsize=(10, 8))
    if labels is not None:
        unique_labels = np.unique(labels)
        for label in unique_labels:
            plt.scatter(
                Y[labels == label, 0], Y[labels == label, 1],
                label=f'Cluster {label}', alpha=0.6
            )
        plt.legend()
    else:
        plt.scatter(Y[:, 0], Y[:, 1], alpha=0.6)
    
    plt.title(title)
    plt.xlabel('TSNE1')
    plt.ylabel('TSNE2')
    plt.show()

# Example usage
# Assuming 'result' is the output of the tsne function and 'clusters' are the cluster labels
# clusters = data['cluster'].values  # or any array of cluster labels
# plot_tsne_results(result, labels=clusters)